In [1]:
import sys
__APPEND_PATHS__ = [
    '/mnt/proj/dsl-common-utils-da1/dsl_helpers',
    '/mnt/proj/dsl-common-utils-da1/ftdu/',
    '/mnt/proj/emob-da1/emil_johansson/helper/',
    '/mnt/proj/emob-da1/emil_johansson/emulate_ess_cycle_doids/',
    '/mnt/proj/emob-da1/mlogtool/v1.1.0/',
]

for p in __APPEND_PATHS__:
    if p not in sys.path:
        sys.path.append(p)
        

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
import time
import os
import re

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# Import functions for emulating doids.P

import build_doids_p4285 as func
# PySpark function
from pyspark.sql import functions as F, Window as W
from pyspark import StorageLevel
from functools import reduce


In [4]:
from cnx_auto_etl import engine
from cnx_dsl_framework import create_engine_dsl_framework
import cnx_spark

No valid kerberos ticket found.  Please enter password to get a new ticket.
········


/usr/local/lib/python3.6/site-packages/sqlalchemy/connectors/pyodbc.py:79: SAWarning: No driver name specified; this is expected by PyODBC when using DSN-less connections
  "No driver name specified; "


In [5]:
import mlogtool
from mlogtool.framework import MLOGFramework
import create_json_config

## Creating a spark instance

In [6]:
list_signals = ["actccsconnectorstatus_ep2_x_cc",
"storage_group",
"vehicle",
"data_filename",
"zip_filename",
"time_abs",
"time_column",
"epsstatus_pt_x_h",
"totalvehicledistancehighres",
"grosscombinationvehicleweight",
"ambientairtemperature",
"frontaxlespeed",
"reqccscharging_ep2_x_h",
"vehiclemode",
"propulsionstatus",
"actsoc_ep1_x_b1",
"actsoc_ep1_x_b2",
"actsoc_ep1_x_b3",
"actsoc_ep1_x_b4",
"actsoc_ep1_x_b5",
"actsoc_ep1_x_b6",
"actsoc_ep4_x_es",
"actcurrent_ep1_x_b1",
"actcurrent_ep1_x_b2",
"actcurrent_ep1_x_b3",
"actcurrent_ep1_x_b4",
"actcurrent_ep1_x_b5",
"actcurrent_ep1_x_b6",
"actpackvoltage_ep1_x_b1",
"actpackvoltage_ep1_x_b2",
"actpackvoltage_ep1_x_b3",
"actpackvoltage_ep1_x_b4",
"actpackvoltage_ep1_x_b5",
"actpackvoltage_ep1_x_b6",
"actsohcap_ep1_x_b1",
"actsohcap_ep1_x_b2",
"actsohcap_ep1_x_b3",
"actsohcap_ep1_x_b4",
"actsohcap_ep1_x_b5",
"actsohcap_ep1_x_b6",
"actmintemp_ep1_x_b1",
"actmintemp_ep1_x_b2",
"actmintemp_ep1_x_b3",
"actmintemp_ep1_x_b4",
"actmintemp_ep1_x_b5",
"actmintemp_ep1_x_b6",
"actmaxtemp_ep1_x_b1",
"actmaxtemp_ep1_x_b2",
"actmaxtemp_ep1_x_b3",
"actmaxtemp_ep1_x_b4",
"actmaxtemp_ep1_x_b5",
"actmaxtemp_ep1_x_b6",
"actavgtemp_ep1_x_b1",
"actavgtemp_ep1_x_b2",
"actavgtemp_ep1_x_b3",
"actavgtemp_ep1_x_b4",
"actavgtemp_ep1_x_b5",
"actavgtemp_ep1_x_b6",
"actstatus_ep1_x_b1",
"actstatus_ep1_x_b2",
"actstatus_ep1_x_b3",
"actstatus_ep1_x_b4",
"actstatus_ep1_x_b5",
"actstatus_ep1_x_b6",
"disconnectstatus_ep1_x_b1",
"disconnectstatus_ep1_x_b2",
"disconnectstatus_ep1_x_b3",
"disconnectstatus_ep1_x_b4",
"disconnectstatus_ep1_x_b5",
"disconnectstatus_ep1_x_b6"]


In [7]:
#Create a list of signals in default name signal_list.json
create_json_config.write_signals_list(list_signals) 

In [8]:
## Only input needed is app_name = "name of session" 
## All parameters are default except vehicle_list which default is empty list for all vehicles
create_json_config.write_config_file(app_name = 'thesis_explore',
     app_size = 'small',topic_list = ['P4285_EMOB'],vehicle_list= ['FM-1156','FH-2367','FM-1156'],
     start_date= '2022-10-01', end_date = '2023-12-31',append_state_flagging = [],sample_time = 1,
     append_sampling = [],name_file = 'config',signal_list_name='signal_list')  

In [9]:
%%time
CONFIG_PATH = os.getcwd()+'/config.json'
# CONFIG_PATH = 'config.json'
OUTPUT_PATH = None
LOCAL_PATH = None
fw = MLOGFramework(CONFIG_PATH, OUTPUT_PATH, LOCAL_PATH)
df, spark = fw.extract_spark_df() #Generates Spark session with name = app_name and spark_df with the data

CPU times: user 456 ms, sys: 78.9 ms, total: 535 ms
Wall time: 44.3 s


## Filtering and Scaling data

In [10]:
%%time
#filtering out the necessary features for preprocessing
import re

df_t = df.select(["totalvehicledistancehighres",
"grosscombinationvehicleweight",
"ambientairtemperature","actsoc_ep1_x_b3","actcurrent_ep1_x_b3","actpackvoltage_ep1_x_b3","actsohcap_ep1_x_b3","actmintemp_ep1_x_b3",
"actmaxtemp_ep1_x_b3", 
"actavgtemp_ep1_x_b3",
"actstatus_ep1_x_b3",
"disconnectstatus_ep1_x_b3","vehiclemode",'driving_cycle','charging_cycle','vehiclemode_encoded',"time_abs","data_filename","zip_filename","vehicle"])

#features list needed to be scaled for temprature features
temp_cols = ["actmintemp_ep1_x_b3",
"actmaxtemp_ep1_x_b3", 
"actavgtemp_ep1_x_b3"]

#making a seperate feature status
df_1 = df_t.withColumn("status",F.concat_ws("_",F.col("driving_cycle"),F.col("charging_cycle"))).drop("driving_cycle").drop("charging_cycle")



CPU times: user 3.47 ms, sys: 436 µs, total: 3.91 ms
Wall time: 160 ms


In [11]:
#scaling temp features

sint16_scale = 3276.7
T0 = -273.15 # use absolute zero as an indicator for when to apply the translation
T1 = 273.15
  
for temp_col in temp_cols:
    df_1 = df_1.withColumn(temp_col, F.when(F.col(temp_col).between(-273.15,273.15), F.col(temp_col)).otherwise(F.when(F.col(temp_col) < T0 ,F.col(temp_col)+sint16_scale)))
    

In [12]:
df_1 = df_1.na.drop(how='any',subset=["totalvehicledistancehighres",
"grosscombinationvehicleweight",
"ambientairtemperature","actsoc_ep1_x_b3","actcurrent_ep1_x_b3","actpackvoltage_ep1_x_b3","actsohcap_ep1_x_b3","actmintemp_ep1_x_b3",
"actmaxtemp_ep1_x_b3", 
"actavgtemp_ep1_x_b3",
"actstatus_ep1_x_b3",
"disconnectstatus_ep1_x_b3","vehiclemode",'vehiclemode_encoded',"time_abs","data_filename","zip_filename","vehicle"])

In [13]:
df_1 = df_1.select("*")\
        .filter((F.col("actsoc_ep1_x_b3").between(0,100)) & (F.col("actsohcap_ep1_x_b3").between(0,100)) & (F.col("ambientairtemperature") < 100) & (F.col("status") == "Driving_NotCharging"))

#have to check the filters for 1year data if needed

## Batching data based on time diff > 600 and changing zip filename

In [14]:
#window function for the procedure
my_window = W.partitionBy().orderBy("time_abs")

In [15]:
w3 = W.partitionBy("vehicle").orderBy(F.col("time_abs"))
df_1 = df_1.withColumn("time_diff", F.lag(F.unix_timestamp("time_abs")).over(w3))\
  .withColumn("time_diff", F.when(F.col("time_diff").isNotNull(), F.unix_timestamp("time_abs")-F.col("time_diff"))\
                         .otherwise(F.lit(0)))

In [16]:
df_1 = df_1.withColumn("name_diff", F.when(F.col("zip_filename") == F.lag(F.col("zip_filename")).over(w3),0).otherwise(1))
        #.withColumn("status_diff",F.when(F.col("status") == F.lag(F.col("status")).over(my_window),0).otherwise(1))

In [17]:
df_1 = df_1.withColumn('Batch', F.when((F.col("name_diff") == 1) | (F.col("time_diff") > 600), 1).otherwise(0))\
        .withColumn("Batch_ID", F.sum(F.col("Batch")).over(my_window))

### Creating individual time stamp difference for each batch 

In [18]:
w3 = W.partitionBy("Batch_ID").orderBy(F.col("time_abs"))
df_diff = df_1.withColumn("batch_time_diff", F.lag(F.unix_timestamp("time_abs")).over(w3))\
  .withColumn("batch_time_diff", F.when(F.col("batch_time_diff").isNotNull(), F.unix_timestamp("time_abs")-F.col("batch_time_diff"))\
                         .otherwise(F.lit(0))).orderBy("Batch_ID","time_abs").drop("Batch").drop("Batch").drop("name_diff").drop("status_diff").drop("status_ohe").drop("vehiclemode_encoded_ohe").drop("time_diff")

## Encoding string of vehicle to int

In [29]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol='vehicle', outputCol='vehicle_int')
indexer.setHandleInvalid("skip")
indexer_fitted = indexer.fit(df_diff)
df_t = indexer_fitted.transform(df_diff)

df_driving = df_t.withColumn("vehicle_int" , F.col("vehicle_int")+1)\
    .drop(F.col("vehicle")).drop(F.col("data_filename")).drop(F.col("status")).drop(F.col("vehiclemode_encoded")).drop(F.col("vehiclemode")).drop(F.col("disconnectstatus_ep1_x_b3")).drop(F.col("zip_filename"))


In [30]:
%%time
df_driving_pd = df_driving.toPandas()


CPU times: user 9.36 s, sys: 37.5 ms, total: 9.4 s
Wall time: 18.9 s


In [54]:
from sklearn.preprocessing import StandardScaler
from math import sqrt

list1 = ['totalvehicledistancehighres', 'grosscombinationvehicleweight',
        'ambientairtemperature', 'actsoc_ep1_x_b3', 'actcurrent_ep1_x_b3',
        'actpackvoltage_ep1_x_b3', 'actsohcap_ep1_x_b3', 'actmintemp_ep1_x_b3',
        'actmaxtemp_ep1_x_b3', 'actavgtemp_ep1_x_b3']


pd_std_df = df_driving_pd.set_index('time_abs')
std_scaler = StandardScaler()

# fit and transform the data
pd_std_df[list1] = std_scaler.fit_transform(pd_std_df[list1])

path = r'/mnt/proj/emob-da1/zz_thesis_work/model_test'

for (gender), group in pd_std_df.groupby(['Batch_ID']):
     group.to_csv(path+f'{gender}.csv', index=False)

In [46]:
len(pd_std_df)

166893

In [52]:
pwd

'/mnt/proj/emob-da1/zz_thesis_work'

In [ ]:
%%time
mean_age, sttdev_age = df_driving.select(F.mean(F.col('totalvehicledistancehighres')), F.stddev(F.col('totalvehicledistancehighres'))).first()
df_std = df_driving.withColumn("totalvehicledistancehighres", (F.col('totalvehicledistancehighres') - mean_age) / sttdev_age)

mean_age, sttdev_age = df_driving.select(F.mean(F.col('actsohcap_ep1_x_b3')), F.stddev(F.col('actsohcap_ep1_x_b3'))).first()
df_std = df_std.withColumn("actsohcap_ep1_x_b3", (F.col('actsohcap_ep1_x_b3') - mean_age) / sttdev_age)

mean_age, sttdev_age = df_driving.select(F.mean(F.col('grosscombinationvehicleweight')), F.stddev(F.col('grosscombinationvehicleweight'))).first()
df_std = df_std.withColumn("grosscombinationvehicleweight", (F.col('grosscombinationvehicleweight') - mean_age) / sttdev_age)

mean_age, sttdev_age = df_driving.select(F.mean(F.col('ambientairtemperature')), F.stddev(F.col('ambientairtemperature'))).first()
df_std = df_std.withColumn("ambientairtemperature", (F.col('ambientairtemperature') - mean_age) / sttdev_age)

mean_age, sttdev_age = df_driving.select(F.mean(F.col('actsoc_ep1_x_b3')), F.stddev(F.col('actsoc_ep1_x_b3'))).first()
df_std = df_std.withColumn("actsoc_ep1_x_b3", (F.col('actsoc_ep1_x_b3') - mean_age) / sttdev_age)

mean_age, sttdev_age = df_driving.select(F.mean(F.col('actcurrent_ep1_x_b3')), F.stddev(F.col('actcurrent_ep1_x_b3'))).first()
df_std = df_std.withColumn("actcurrent_ep1_x_b3", (F.col('actcurrent_ep1_x_b3') - mean_age) / sttdev_age)

mean_age, sttdev_age = df_driving.select(F.mean(F.col('actpackvoltage_ep1_x_b3')), F.stddev(F.col('actpackvoltage_ep1_x_b3'))).first()
df_std = df_std.withColumn("actpackvoltage_ep1_x_b3", (F.col('actpackvoltage_ep1_x_b3') - mean_age) / sttdev_age)

mean_age, sttdev_age = df_driving.select(F.mean(F.col('actmintemp_ep1_x_b3')), F.stddev(F.col('actmintemp_ep1_x_b3'))).first()
df_std = df_std.withColumn("actmintemp_ep1_x_b3", (F.col('actmintemp_ep1_x_b3') - mean_age) / sttdev_age)

mean_age, sttdev_age = df_driving.select(F.mean(F.col('actmaxtemp_ep1_x_b3')), F.stddev(F.col('actmaxtemp_ep1_x_b3'))).first()
df_std = df_std.withColumn("actmaxtemp_ep1_x_b3", (F.col('actmaxtemp_ep1_x_b3') - mean_age) / sttdev_age)

mean_age, sttdev_age = df_driving.select(F.mean(F.col('actavgtemp_ep1_x_b3')), F.stddev(F.col('actavgtemp_ep1_x_b3'))).first()
df_std = df_std.withColumn("actavgtemp_ep1_x_b3", (F.col('actavgtemp_ep1_x_b3') - mean_age) / sttdev_age)

## Saving the dataframe as parquet batch-wise

In [ ]:
df_std.repartition(1).write.partitionBy("Batch_ID").parquet("Batch_data_2022")
#df_std.write.partitionBy('Batch_ID').format("csv").save("csv_test")


In [ ]:
!hdfs dfs -copyToLocal Batch_data_2022 /mnt/proj/emob-da1/zz_thesis_work


## Creating pandas df for EDA over batches

In [ ]:
index = ['totalvehicledistancehighres',
 'grosscombinationvehicleweight',
 'ambientairtemperature',
 'actsoc_ep1_x_b3',
 'actcurrent_ep1_x_b3',
 'actpackvoltage_ep1_x_b3',
 'actsohcap_ep1_x_b3',
 'actmintemp_ep1_x_b3',
 'actmaxtemp_ep1_x_b3',
 'actavgtemp_ep1_x_b3',
 'actstatus_ep1_x_b3',
 'time_abs',
 'zip_filename',
 'Batch_ID',
 'batch_time_diff',
 'vehicle_int']

pd_long = pd.read_parquet('/mnt/proj/emob-da1/zz_thesis_work/Batch_data_2022/Batch_ID=33',engine='fastparquet')


In [ ]:

g_cols = ["actmintemp_ep1_x_b3",
"actmaxtemp_ep1_x_b3", 
"actavgtemp_ep1_x_b3","ambientairtemperature","actsoc_ep1_x_b3","actcurrent_ep1_x_b3","actpackvoltage_ep1_x_b3","actsohcap_ep1_x_b3"]

pd_g = pd_long[g_cols]

fig, a = plt.subplots(2, 4, figsize=(30, 10), tight_layout=True)
pd_g.plot(ax=a, subplots=True, rot=60);

In [ ]:
pd_long[pd_long['actstatus_ep1_x_b3'] == 3.0]


In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

#perform Granger-Causality test
grangercausalitytests(df[['egg', 'chicken']], maxlag=[3])

In [ ]:
df_temp = df_t[df_t.charging_cycle == 'Charging']
fig, ax = plt.subplots(1,1, figsize = (20,10))

signal = 'actsoc_ep1_x_b1'
color_list = ['black', 'red', 'magenta']
for k, label in enumerate(df_temp.status.unique()):
    mask = (df_t['status'] == label) & (df_temp.actcurrent_ep1_x_b1 > -150)
    ax.scatter(df_temp[mask].time_abs, df_temp[mask].actcurrent_ep1_x_b1, c = color_list[k])
    ax2 = ax.twinx()
    ax2.scatter(df_temp[mask].time_abs, df_temp[mask][signal], label = label)
ax.legend()


In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
pd_pca = pca.fit(pd_long)

In [ ]:
spark.stop()

In [ ]:
pd_long = pd_long.drop(columns="zip_filename")

In [ ]:
pd_pca.explained_variance_ratio_

In [ ]:
plt.plot(np.cumsum(pd_pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')



In [ ]:
num_components = 4
pca = PCA(num_components)
X_pca = pca.fit_transform(pd_long) # fit and reduce dimension

In [ ]:
pd.DataFrame(pca.components_, columns = pd_long.columns)

In [ ]:
cp -r ./Batch_data_2022/Batch_ID=10 ./Batch_data_2022/Batch_ID=105 ./diff_batches

In [ ]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

list1 = ['totalvehicledistancehighres', 'grosscombinationvehicleweight',
        'ambientairtemperature', 'actsoc_ep1_x_b3', 'actcurrent_ep1_x_b3',
        'actpackvoltage_ep1_x_b3', 'actsohcap_ep1_x_b3', 'actmintemp_ep1_x_b3',
        'actmaxtemp_ep1_x_b3', 'actavgtemp_ep1_x_b3']


df1 = pd.read_parquet('/mnt/proj/emob-da1/zz_thesis_work/Batch_data_2022/Batch_ID=31',engine='fastparquet')
df2 = pd.read_parquet('/mnt/proj/emob-da1/zz_thesis_work/Batch_data_2022/Batch_ID=11',engine='fastparquet')
df3 = pd.read_parquet('/mnt/proj/emob-da1/zz_thesis_work/Batch_data_2022/Batch_ID=237',engine='fastparquet')
df4 = pd.read_parquet('/mnt/proj/emob-da1/zz_thesis_work/Batch_data_2022/Batch_ID=479',engine='fastparquet')
df5 = pd.read_parquet('/mnt/proj/emob-da1/zz_thesis_work/Batch_data_2022/Batch_ID=546',engine='fastparquet')
df6 = pd.read_parquet('/mnt/proj/emob-da1/zz_thesis_work/Batch_data_2022/Batch_ID=382',engine='fastparquet')

dfs = {"df1" : df1, "df2": df2, "df3" : df3, "df4" : df4, "df5" : df5, "df6" : df6}

# fig = make_subplots(rows = 2,cols = 1)

# fig.append_trace(go.Scatter(x=df1.time_abs, y=df1[list1], name="a"),row = 1, col=1)
# fig.append_trace(go.Scatter(x=df2.time_abs, y=df2[list1], name="b"),row = 2, col=1)


# import matplotlib.pyplot as plt

# fig, axes = plt.subplots(nrows=2, ncols=2)

# df1.plot(ax=axes[0,0])
# df2.plot(ax=axes[0,1])

# fig.show()






In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px


list1 = ['totalvehicledistancehighres', 'grosscombinationvehicleweight',
        'ambientairtemperature', 'actsoc_ep1_x_b3', 'actcurrent_ep1_x_b3',
        'actpackvoltage_ep1_x_b3', 'actsohcap_ep1_x_b3', 'actmintemp_ep1_x_b3',
        'actmaxtemp_ep1_x_b3', 'actavgtemp_ep1_x_b3']

# fig = make_subplots(rows = 3,cols = 2)

# fig = px.line(df1, x=df1.time_abs, y=list1,row=1,col=1)
# fig = px.line(df1, x=df2.time_abs, y=list1,row=2,col=1)
# fig = px.line(df1, x=df3.time_abs, y=list1,row=3,col=1)
# fig = px.line(df1, x=df4.time_abs, y=list1,row=1,col=2)
# fig = px.line(df1, x=df5.time_abs, y=list1,row=2,col=2)
# fig = px.line(df1, x=df6.time_abs, y=list1,row=3,col=2)


# fig.show()

fig = make_subplots(rows=2, cols=1)
trace1 = px.line(df1, x='time_abs', y=list1)
trace2 = px.line(df2, x='time_abs', y=list1)

fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=2, col=1)
fig.show()


In [ ]:
df1